### Covid-19 Live Data

In [ ]:
import pandas as pd

allData = pd.read_excel("https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide.xlsx")

allData.head()

#### Filtering Criteria

In [ ]:
print(allData.geoId.unique()) #sales territories
geoId = input("choose a territory, or a for all: ")

filtered_df = allData.loc[allData['geoId'] == geoId] #region filter

filtered_df.head()

In [ ]:
filtered_df.dtypes

In [ ]:
#set MonthYear as index
filtered_df.set_index('dateRep', inplace=True)

#plot cases and deaths

#filter plot data only
plot_df=filtered_df[[ "cases","deaths"]]
plot_df.head()

ax = plot_df.plot(figsize=(14,6), lw=2, style='k-')